In [1]:
from typing import Dict, Tuple
import torchvision
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import models, transforms
from torchvision.utils import save_image, make_grid
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from torchvision import datasets, transforms
import numpy as np
from IPython.display import HTML
from diffusion_utilities import *

In [2]:
from torchvision import datasets
transformerr = transforms.Compose([
    transforms.ToTensor(),  # Convert to tensor
])

# Time mapping

In [3]:
step = list(range(1,1001))
beta = dict()
for t in step:
    beta[t] = round(t * 0.0199 / 999 + (0.02 - 0.0199 / 999 * 1000), 15) # beta = at + b
alpha = dict()
alpha[1] = 1 - beta[1]
for t in step[1:]:
    alpha[t] = alpha[t-1] * (1 - beta[t])


In [4]:
import math
T = dict()
for t in step:
    T[t] = -1/2 * math.log(alpha[t])

In [10]:
T[0] = 0

In [11]:
t = np.array([T[x] for x in range(0,1001, 20)])

In [12]:
t

array([0.00000000e+00, 2.89287678e-03, 9.77168452e-03, 2.06380126e-02,
       3.54934518e-02, 5.43395939e-02, 7.71780322e-02, 1.04010361e-01,
       1.34838177e-01, 1.69663075e-01, 2.08486656e-01, 2.51310518e-01,
       2.98136262e-01, 3.48965490e-01, 4.03799807e-01, 4.62640815e-01,
       5.25490122e-01, 5.92349335e-01, 6.63220063e-01, 7.38103914e-01,
       8.17002501e-01, 8.99917436e-01, 9.86850333e-01, 1.07780281e+00,
       1.17277647e+00, 1.27177295e+00, 1.37479385e+00, 1.48184081e+00,
       1.59291544e+00, 1.70801936e+00, 1.82715420e+00, 1.95032159e+00,
       2.07752314e+00, 2.20876049e+00, 2.34403528e+00, 2.48334912e+00,
       2.62670365e+00, 2.77410051e+00, 2.92554132e+00, 3.08102773e+00,
       3.24056138e+00, 3.40414389e+00, 3.57177692e+00, 3.74346210e+00,
       3.91920107e+00, 4.09899548e+00, 4.28284698e+00, 4.47075721e+00,
       4.66272782e+00, 4.85876046e+00, 5.05885677e+00])

In [5]:
T[195] #collapse time experimental

0.19840579691312765

In [8]:
mapped_time = min(T, key=lambda k: abs(T[k]-0.278))
mapped_time

232

# Speciation time

In [19]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transformerr)

Files already downloaded and verified


In [21]:
torch.min(trainset[0][0])

tensor(0.)

In [22]:
mask_1 = torch.Tensor(trainset.targets) == 1
mask_2 = torch.Tensor(trainset.targets) == 7
mask = mask_1 | mask_2
trainset = torch.utils.data.Subset(trainset, torch.where(mask)[0])

In [34]:
trainset[0][0][0].shape

torch.Size([32, 32])

In [23]:
dataset = torch.Tensor()

In [41]:
# Initialize an empty list to store tensors
tensor_list = []

# Loop through the dataset and collect the tensors
for x in range(len(trainset)):
    tensor_list.append(torch.unsqueeze(trainset[x][0][0], 0))
    

# Concatenate all tensors at once
dataset = torch.cat(tensor_list, dim=0)

print(dataset.shape)

torch.Size([10000, 32, 32])


In [44]:
dataset -= torch.mean(dataset)

In [46]:
row = []
for i in range(len(dataset)):
    row.append(dataset[i].flatten().tolist())

In [47]:
torch.Tensor(row).shape

torch.Size([10000, 1024])

In [48]:
row = torch.transpose(torch.Tensor(row),0,1)

In [49]:
C = torch.cov(torch.tensor(row))

/var/folders/cd/sr6rtt2j1rqdn0jdyblt3ztm0000gn/T/ipykernel_57070/3888172521.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  C = torch.cov(torch.tensor(row))


In [50]:
C.shape

torch.Size([1024, 1024])

In [51]:
with torch.no_grad():
    L, _ = torch.linalg.eigh(C, UPLO="L")

In [52]:
L[-1]

tensor(16.5853)

In [53]:
t_s = 0.5 * np.log(L[-1])
t_s

tensor(1.4043)

In [54]:
mapped_time = min(T, key=lambda k: abs(T[k]-t_s))
mapped_time

526

# Collapse time

In [18]:
# MNIST
from torchvision import datasets, transforms
dataset = datasets.MNIST('~/.pytorch/MNIST_data/', train=True, download=True, transform=transform)

In [20]:
def first_n (N, mask):
    n = 0
    for i in range(len(mask)):
        if mask[i]:
            n += 1
        if n == N:
            for j in range(i+1, len(mask)):
                mask[j] = False
            break
    return mask

In [21]:
mask_1 = dataset.targets == 0
first_n(100, mask_1)
mask_2 = dataset.targets == 7
first_n(100, mask_2)
mask = mask_1 | mask_2
dataset = torch.utils.data.Subset(dataset, torch.where(mask)[0])


In [39]:
a = torch.Tensor()
for x in range(len(dataset)):
    a = torch.cat((a, dataset[x][0]), dim=0)

In [69]:
@torch.no_grad
def P(t, x):
    d = 28 * 28
    return torch.sum(1/len(dataset) * 1/(2*torch.pi*(1-torch.e**(0-2*t)))**(d/2) * torch.exp(-1/2* (x-a*torch.e**(0-t))**2 / (1-torch.e**(0-2*t))))

In [70]:
P(1,a[1])

tensor(0.)

In [66]:
torch.sum(a, dim=0).shape

torch.Size([28, 28])